# Create a training routine to shuffle and make sure every features is capture

In [ ]:
%%time

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from sklearn.preprocessing import normalize

plt.rcParams['figure.figsize'] = 5.0, 4.0

from pyts.transformation import GADF,GASF
from sklearn.preprocessing import normalize

import uproot
import torch
from torch.utils.data import DataLoader
from torch.autograd import Variable

from scipy.cluster.hierarchy import dendrogram, linkage
from scipy.cluster.hierarchy import fcluster

import fastcluster as fc

# Load data and clean data

In [ ]:
test = uproot.open("sig6.root")["tree"]
Phonon = test.array("HeightHeat")
Photon = test.array("HeightLight")
eventid = test.array("EventNumber")
print(Photon[1].shape,Phonon.shape,eventid)
del test

In [ ]:
%%time
length = len(Photon)

ped_heat = np.mean(Phonon[:,50:350], axis=1)
ped_light = np.mean(Photon[:,50:350], axis=1)

for i,j in enumerate(Phonon):
        Phonon[i] = Phonon[i] - ped_heat[i]
        Photon[i] = Photon[i] - ped_light[i]

start_heat = np.argmax(Phonon[:,350:450], axis=1)
# start_light = np.argmax(Photon[:,350:450], axis=1)
start_heat +=350
# start_light +=350

# energy_heat = Phonon[:,800]
# energy_heat2 = np.amax(Phonon[:,360:420], axis=1)
energy_light = np.amax(Photon[:,360:420], axis=1)

energy_heat = np.empty(length)
energy_heat1 = np.empty(length)
energy_heat2 = np.empty(length)

for i in range(length):
    energy_heat[i] = Phonon[i,int(start_heat[i]+400)]
    energy_heat1[i] = Phonon[i,int(start_heat[i]+370)]
    energy_heat2[i] = Phonon[i,int(start_heat[i]+350)]

ratio = energy_light/energy_heat
ratio1 = energy_light/energy_heat1
ratio2 = energy_light/energy_heat2

# Shuffler and flipper

# Transform data into suitable training format

In [ ]:
# Normalizaion
norm_heat = np.amax(Phonon,axis=1)
norm_light = np.amax(Photon,axis=1)

for i,j in enumerate(Phonon):
    if norm_heat[i] > norm_light[i]:
        Phonon[i] = Phonon[i]/norm_heat[i]
        Photon[i] = Photon[i]/norm_heat[i]
    if norm_light[i] > norm_heat[i]:
        Phonon[i] = Phonon[i]/norm_light[i]
        Photon[i] = Photon[i]/norm_light[i]

In [ ]:
dataset = np.zeros((len(Phonon),2,2000))
for i in range(len(Phonon)):
    dataset[i][0] = Phonon[i]
    dataset[i][1] = Photon[i]

del Phonon,Photon

In [ ]:
BATCH = 10000
train_loader = DataLoader(dataset=dataset,
                         batch_size=BATCH,
                         shuffle=False,
                         num_workers=2)

n_batches = int(len(dataset)/BATCH)
del dataset

# 2 channel Auto-encoder network

In [ ]:
from torch import nn
from torch.autograd import Variable
import torch.nn.functional as F


class Autoencoder_2c(nn.Module):
    def __init__(self):
        super(Autoencoder_2c, self).__init__()        
#         self.norm = nn.BatchNorm1d(1)
        self.cv1 = nn.Conv1d(2, 32, kernel_size=8, stride=4, padding=0)
        self.pl1 = nn.MaxPool1d(2, stride=4)
        self.cv21 = nn.Conv1d(32, 32, kernel_size=8, stride=4, padding=0)
        self.pl21 = nn.MaxPool1d(2, stride=4)
        self.cv22 = nn.Conv1d(32, 32, kernel_size=8, stride=4, padding=0)
        self.pl22 = nn.MaxPool1d(2, stride=4)
                
        self.ct1 = nn.ConvTranspose1d(32, 32, kernel_size=8, stride=4, padding=2)
        self.up1 = nn.Upsample(scale_factor=2, mode='nearest')
        self.ct2 = nn.ConvTranspose1d(32, 32, kernel_size=6, stride=4, padding=4)
        self.up2 = nn.Upsample(scale_factor=2, mode='nearest')
        self.ct3 = nn.ConvTranspose1d(32, 2, kernel_size=8, stride=4,padding=2)

    def encoder(self, x):
#         h0 = self.norm(x)
        h1 = F.tanh(self.pl1(self.cv1(x)))
        return self.pl21(self.cv21(h1)), self.pl22(self.cv22(h1))
    
    def reparameterise(self, mu, logvar):
        std = logvar.mul(0.5).exp_()
        if torch.cuda.is_available():
            eps = torch.cuda.FloatTensor(std.size()).normal_()
        else:
            eps = torch.FloatTensor(std.size()).normal_()
        eps = Variable(eps)
        return eps.mul(std).add_(mu)
    
    def decoder(self, z):
        h3 = F.leaky_relu(self.ct1(z))
        h3 = self.up1(h3)
        h3 = F.leaky_relu(self.ct2(h3))
        h3 = self.up2(h3)
        return F.leaky_relu(self.ct3(h3))
    
    def forward(self,inputs):
        mu, logvar = self.encoder(inputs)
        z = self.reparameterise(mu, logvar)
        return self.decoder(z), mu, logvar
print("Autoencoder")

GEN = Autoencoder_2c()
print(GEN)

criterion = nn.L1Loss()
optimizer = torch.optim.Adam(GEN.parameters(), lr=0.001, weight_decay=1e-5)

for parameter in GEN.parameters():
    print(parameter.size())

# Training the 2 channel auto-encoder

In [ ]:
%%time
N_EPOCHS = 10
# allow for manual keyboard interrupt
try: 
    # loop through epochs
    for epoch in range(N_EPOCHS):
        for batch_number, waveform in enumerate(train_loader):
            
            target = waveform.float()
            outputs = GEN(waveform.float())

            loss = criterion(outputs[0], target)
            
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        
            if (batch_number +1)%5 == 0:
                print("Epoch[%d/%d], Step[%d/%d], loss=%.6f"
                      %(epoch+1,
                        N_EPOCHS,
                        batch_number+1,
                        n_batches,
                        losTrues.data[0] ))
except KeyboardInterrupt:
    print('Training ended early.')

In [ ]:
torch.save(GEN.state_dict(),"dual.pkl")